<a href="https://colab.research.google.com/github/Anil1331/EIP3-Assignments/blob/master/MNIST_3a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from keras.datasets import mnist
import keras
import warnings
num_classes = 10
(x_train, y_train), (x_test, y_test) = mnist.load_data()
img_height, img_width = x_train.shape[1],x_train.shape[2]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
from keras.preprocessing.image import ImageDataGenerator
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train/= 255
x_test/= 255
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, Conv2D, SeparableConv2D
from keras.callbacks import Callback, ModelCheckpoint
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(SeparableConv2D(32, 3, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(SeparableConv2D(64, 3, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(32, 1, 1))
model.add(SeparableConv2D(128, 3, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(10, 1))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Convolution2D(10, 4))
model.add(Flatten())
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()


class EarlyStoppingByAccuracy(Callback):
    def __init__(self, monitor='val_acc', mode='max', value=0.98, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)
        
        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

callbacks = [
    EarlyStoppingByAccuracy(monitor='val_acc', value=0.992, verbose=1),
    ModelCheckpoint(filepath='/tmp/weights.hdf5', monitor='val_loss', save_best_only=True, verbose=0),
]          



image_gen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.5,
        zoom_range=(0.9, 1.1),
        horizontal_flip=False,
        vertical_flip=False,
        fill_mode='constant',
        cval=0)

#training the image preprocessing
image_gen.fit(x_train, augment=True)
'''
model.fit(x_train, y_train,
                    batch_size=256,
                    epochs=30,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks)                    
'''                   
model.fit_generator(image_gen.flow(x_train, y_train, batch_size=256),
          epochs=30,
          steps_per_epoch=len(x_train)/(256),
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=callbacks)          

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(32, (3, 3))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(64, (3, 3))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(128, (3, 3))`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_127 (Conv2D)          (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_113 (Bat (None, 26, 26, 32)        128       
_________________________________________________________________
activation_134 (Activation)  (None, 26, 26, 32)        0         
_________________________________________________________________
separable_conv2d_49 (Separab (None, 24, 24, 32)        1344      
_________________________________________________________________
batch_normalization_114 (Bat (None, 24, 24, 32)        128       
_________________________________________________________________
activation_135 (Activation)  (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 12, 12, 32)        0         
__________